# Adding a new Session 

In [26]:
%load_ext autoreload
%autoreload 2

import datajoint as dj

dj.config["enable_python_native_blobs"] = True
dj.config['schema_name'] = "nnfabrik_color_mei"
schema = dj.schema("nnfabrik_color_mei")

from matplotlib import pyplot as plt
from torch import load

from nnfabrik.main import *
import nnfabrik
from nnfabrik import main, builder
import os
from os import listdir
from os.path import isfile, join
import torch
import matplotlib.pyplot as plt
import numpy as np
from mlutils.measures import corr

from mlutils.data.datasets import StaticImageSet, FileTreeDataset
from nndichromacy.tables.from_nnfabrik import TrainedModel

from dataport.bcm.static import PreprocessedMouseData
from nndichromacy.datasets.utility import add_h5_to_preprocessed_table

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# First of all: Copy the h5 file to the same path as this notebook

In [11]:
keys = dict(animal_id=23656, session=27, scan_idx=8)
extended_comment = 'Mouse color experiment. Stimuli: Imagenet - High MSE. B/G/Dependent'
dataset_comment = '23656-27-8'
notebook_path = os.getcwd()

In [8]:
filenames = add_h5_to_preprocessed_table(notebook_path, keys, extended_comment)

# Insert entry into nnfabriks dataset table

In [21]:
dataset_config = {'paths': filenames, 
                  'normalize': True, 
                  'include_behavior': False, 
                  'batch_size': 128, 
                  'file_tree': True}
dataset_fn = 'nndichromacy.datasets.static_loaders'

key = Dataset().add_entry(dataset_fn, dataset_config, dataset_comment=dataset_comment)
dataset_hash = key["dataset_hash"]


### test building the dataloader

In [22]:
test_build = (Dataset() & f"dataset_hash='{dataset_hash}'").get_dataloader(seed=1000)
data_key = list(test_build["train"].keys())[0]
print("Dataloader built successfully. Example image batch shape: ", next(iter(test_build["train"][data_key])).images.shape)

static23656-27-8-preproc0 exists already. Not unpacking static23656-27-8-preproc0.zip
Dataloader built successfully. Example image shape:  torch.Size([128, 2, 36, 64])


# Training the Model with 5 seeds, using the standard model

In [23]:
model_hash = 'a09d55344b5793137349b8ca5d17d46d'
Model & dict(model_hash=model_hash)

model_fn name of the model function,model_hash hash of the model configuration,model_config model configuration to be passed into the function,model_fabrikant Name of the contributor that added this entry,model_comment short description,model_ts UTZ timestamp at time of insertion
nndichromacy.models.se_core_full_gauss_readout,a09d55344b5793137349b8ca5d17d46d,=BLOB=,kwilleke,full_gauss_cortex,2020-05-17 19:32:45


In [24]:
trainer_hash = '0d06f037501e129d11aa288d8f22788f'
Trainer & dict(trainer_hash=trainer_hash)

trainer_fn name of the Trainer loader function,trainer_hash hash of the configuration object,trainer_config training configuration object,trainer_fabrikant Name of the contributor that added this entry,trainer_comment short description,trainer_ts UTZ timestamp at time of insertion
nndichromacy.training.standart_trainer,0d06f037501e129d11aa288d8f22788f,=BLOB=,kwilleke,standard_trainer,2020-05-17 22:32:21


In [25]:
key = dict(dataset_hash=dataset_hash, model_hash=model_hash, trainer_hash=trainer_hash)

In [ ]:
TrainedModel.populate(key, display_progress=True, reserve_jobs=True)